In [ ]:

# FINAL PROJECT REPLICATION:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from scipy import linalg, optimize
from sklearn.covariance import LedoitWolf

# Reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cpu


In [13]:

# 1. DATA LOADING

def get_real_data():
    tickers = [
        "GOOGL", "AAPL", "MSFT", "MU", "INTC", "CSCO", "BIDU", "ADBE", "TXN", "AVGO",
        "QCOM", "AMT", "HPQ", "IBM", "ORCL", "NTAP", "GLW", "AMAT", "AMZN", "LOGI"
    ]
    print("Downloading data for 20 Tech Stocks (2015-2017)...")
    data = yf.download(tickers, start="2015-06-01", end="2017-06-01")["Close"]
    
    # Log Returns
    returns = np.log(data / data.shift(1)).dropna()
    # Train (Year 1) / Test (Year 2)
    split_idx = 252
    train_data = returns.iloc[:split_idx]
    test_data = returns.iloc[split_idx:]
    
    return train_data, test_data

In [ ]:


# 2. DEEP LEARNING MODEL (Student-t AEM)

# --- Math Utils ---
def students_t_nll(x, ln_df, loc, ln_scale, dim=1):
    """Robust Student-t Negative Log Likelihood"""
    nll = torch.lgamma((torch.exp(ln_df) + 1) / 2) - torch.lgamma(torch.exp(ln_df) / 2)
    nll += (ln_scale - np.log(np.pi) - ln_df) / 2
    nll -= (torch.exp(ln_df) + 1) * torch.log(1 + (torch.exp(ln_scale) * ((x - loc) ** 2) / torch.exp(ln_df))) / 2
    return torch.sum(-1 * nll, dim=dim)

def reparameterize(mu, ln_var):
    std = torch.exp(0.5 * ln_var)
    eps = torch.randn_like(std)
    return mu + std * eps

def gaussian_kl(mu, ln_var, dim=1):
    return torch.sum(-0.5 * (1 + ln_var - mu.pow(2) - ln_var.exp()), dim=dim)

# --- Network ---
class StudentsTVAE(nn.Module):
    def __init__(self, n_in, n_latent=10, n_h=64):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(n_in, n_h), nn.Tanh(),
            nn.Linear(n_h, n_latent * 2) # mu, ln_var
        )
        self.dec_h = nn.Sequential(
            nn.Linear(n_latent, n_h), nn.Tanh()
        )
        self.dec_params = nn.Linear(n_h, n_in * 3) # df, loc, scale

    def forward(self, x):
        h_enc = self.enc(x)
        mu, ln_var = h_enc.chunk(2, dim=1)
        z = reparameterize(mu, ln_var)
        h_dec = self.dec_h(z)
        params = self.dec_params(h_dec)
        ln_df, loc, ln_scale = params.chunk(3, dim=1)
        
        RE = students_t_nll(x, ln_df, loc, ln_scale)
        KL = gaussian_kl(mu, ln_var)
        return RE + KL, loc, ln_scale

# AEM Cleaner
def clean_aem(returns, n_latent=10, epochs=1000):
    """
    Advanced AEM: Uses 'Structured Trace Preservation'.
    Keeps n_latent=10 (Smart) but uses the Model's learned noise profile 
    to create a safer, more accurate risk model.
    """
    # 1. SCALING
    scaler_mean = returns.mean(axis=0).values
    scaler_std = returns.std(axis=0).values
    scaler_std[scaler_std == 0] = 1.0
    
    scaled_returns = (returns - scaler_mean) / scaler_std
    data = torch.FloatTensor(scaled_returns.values).to(device)
    
    # 2. MODEL SETUP
    model = StudentsTVAE(n_in=returns.shape[1], n_latent=n_latent).to(device)
    opt = optim.Adam(model.parameters(), lr=0.002)
    
    # 3. TRAINING
    model.train()
    for i in range(epochs):
        opt.zero_grad()
        loss_vector, _, _ = model(data)
        loss = loss_vector.mean()
        loss.backward()
        opt.step()
        
    # 4. EXTRACTION
    model.eval()
    with torch.no_grad():
        _, loc, ln_scale = model(data)
        
        # Signal (Location)
        clean_scaled = loc.cpu().numpy()
        
        # Noise Shape (Student-t Scale)
        # We take the average learned scale across the timeframe
        avg_ln_scale = ln_scale.mean(dim=0).cpu().numpy()

    # 5. RESCALING SIGNAL
    clean_returns = (clean_scaled * scaler_std) + scaler_mean
    clean_returns_df = pd.DataFrame(clean_returns, index=returns.index, columns=returns.columns)
    
    # 6. STRUCTURED TRACE PRESERVATION
    
    # A. Calculate Energies
    cov_signal = clean_returns_df.cov()
    trace_total = np.trace(returns.cov().values) # True total variance
    trace_signal = np.trace(cov_signal.values)   # Variance captured by Autoencoder
    missing_energy = max(trace_total - trace_signal, 1e-4) # Variance we need to put back
    
    # B. Calculate Learned Noise Shape
    noise_shape = (np.exp(avg_ln_scale) * scaler_std) ** 2
    
    # C. Normalize Noise to fill the Missing Energy
    scale_factor = missing_energy / np.sum(noise_shape)
    final_noise_diag = noise_shape * scale_factor
    
    # 7. FINAL COVARIANCE
    cov_final = cov_signal + np.diag(final_noise_diag)
    
    return cov_final

In [ ]:


# 3. BASELINE CLEANERS
def clean_rmt(returns, k_min=3):
    T, N = returns.shape
    cov = returns.cov().values
    vols = np.sqrt(np.diag(cov))
    D_inv = np.diag(1.0 / vols)
    corr = D_inv @ cov @ D_inv
    
    evals, evecs = linalg.eigh(corr)
    # Sort
    idx = evals.argsort()[::-1]
    evals = evals[idx]
    evecs = evecs[:, idx]
    
    q = N / T
    lambda_max = (1 + np.sqrt(q))**2
    signal_mask = evals > lambda_max
    signal_mask[:k_min] = True # Force market factors
    
    if (~signal_mask).sum() > 0:
        evals[~signal_mask] = evals[~signal_mask].mean()
        
    corr_clean = evecs @ np.diag(evals) @ evecs.T
    np.fill_diagonal(corr_clean, 1.0)
    D = np.diag(vols)
    return pd.DataFrame(D @ corr_clean @ D, index=returns.columns, columns=returns.columns)

def clean_lw(returns):
    lw = LedoitWolf()
    lw.fit(returns)
    return pd.DataFrame(lw.covariance_, index=returns.columns, columns=returns.columns)

In [ ]:


# 4. PORTFOLIO BACKTESTING
def optimize_portfolio(cov_matrix, expected_returns):
    num_assets = len(expected_returns)
    
    def neg_sharpe(weights):
        ret = np.dot(weights, expected_returns)
        vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -ret / vol

    # Constraints: Sum weights = 1, Allow shorting [-1, 1] per paper logic
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((-1.0, 1.0) for _ in range(num_assets))
    init_guess = num_assets * [1. / num_assets,]
    
    result = optimize.minimize(neg_sharpe, init_guess, method='SLSQP', 
                               bounds=bounds, constraints=constraints, tol=1e-10)
    return result.x

def backtest(weights, test_returns):
    port_ret = test_returns.dot(weights)
    ann_ret = port_ret.mean() * 252
    ann_vol = port_ret.std() * np.sqrt(252)
    sharpe = ann_ret / ann_vol
    return ann_ret, ann_vol, sharpe, port_ret

# 5. RUN THE EXPERIMENT
train_df, test_df = get_real_data()
print("\nCleaning Matrices...")
cov_bench = train_df.cov()
cov_rmt = clean_rmt(train_df)
cov_lw = clean_lw(train_df)
# Increase latent factors to 10 for better signal capture in 20 stocks
cov_aem = clean_aem(train_df, n_latent=10, epochs=1000)
print("Optimizing Portfolios...")
mu = train_df.mean()
w_bench = optimize_portfolio(cov_bench.values, mu.values)
w_rmt = optimize_portfolio(cov_rmt.values, mu.values)
w_lw = optimize_portfolio(cov_lw.values, mu.values)
w_aem = optimize_portfolio(cov_aem.values, mu.values)

# Results
r_bench = backtest(w_bench, test_df)
r_rmt = backtest(w_rmt, test_df)
r_lw = backtest(w_lw, test_df)
r_aem = backtest(w_aem, test_df)

print(f"\n{'Portfolio':<15} {'Return':<10} {'Vol':<10} {'Sharpe':<10}")
print("-" * 50)
print(f"{'Benchmark':<15} {r_bench[0]:.2%}     {r_bench[1]:.2%}     {r_bench[2]:.4f}")
print(f"{'RMT':<15} {r_rmt[0]:.2%}     {r_rmt[1]:.2%}     {r_rmt[2]:.4f}")
print(f"{'LWS':<15} {r_lw[0]:.2%}     {r_lw[1]:.2%}     {r_lw[2]:.4f}")
print(f"{'AEM (Ours)':<15} {r_aem[0]:.2%}     {r_aem[1]:.2%}     {r_aem[2]:.4f}")

# Diagnostics
print("\n--- Diagnostics ---")
print(f"AEM Condition Number: {np.linalg.cond(cov_aem):.2f} (Target < 200)")
print(f"RMT Condition Number: {np.linalg.cond(cov_rmt):.2f}")

# Plot
plt.figure(figsize=(10, 6))
(1 + r_bench[3]).cumprod().plot(label='Benchmark', color='gray', linestyle='--')
(1 + r_rmt[3]).cumprod().plot(label='RMT', color='green')
(1 + r_aem[3]).cumprod().plot(label='AEM (Student-t)', color='red', linewidth=2)
plt.title("Cumulative Wealth")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

[*****                 10%                       ]  2 of 20 completed

[*********************100%***********************]  20 of 20 completed



Cleaning Matrices...
